In [1]:
from functools import partial
from random import expovariate, sample

import numpy as np
import simpy

from ns.packet.dist_generator import DistPacketGenerator
from ns.packet.sink import PacketSink
from ns.switch.switch import SimplePacketSwitch
from ns.switch.switch import FairPacketSwitch
from ns.topos.fattree import build as build_fattree
from ns.topos.utils import generate_fib, generate_flows

In [2]:
env = simpy.Environment()

# Hyper params
n_flows = 100
k = 4
pir = 100000
buffer_size = 1000
mean_pkt_size = 100.0

In [3]:
# Generate the topology
ft = build_fattree(k)

In [4]:
# Setup the hosts
hosts = set()
for n in ft.nodes():
    if ft.nodes[n]['type'] == 'host':
        hosts.add(n)


# Generate the flows
all_flows = generate_flows(ft, hosts, n_flows)
size_dist = partial(expovariate, 1.0 / mean_pkt_size)

for fid in all_flows:
    arr_dist = partial(expovariate, 1 + np.random.rand())

    pg = DistPacketGenerator(env,
                             f"Flow_{fid}",
                             arr_dist,
                             size_dist,
                             flow_id=fid)
    ps = PacketSink(env)

    all_flows[fid].pkt_gen = pg
    all_flows[fid].pkt_sink = ps

ft = generate_fib(ft, all_flows)

In [5]:
n_classes_per_port = 4
weights = {c: 1 for c in range(n_classes_per_port)}


def flow_to_classes(f_id, n_id=0, fib=None):
    return (f_id + n_id + fib[f_id]) % n_classes_per_port

In [6]:
# Setup the switches
for node_id in ft.nodes():
    node = ft.nodes[node_id]
    flow_classes = partial(flow_to_classes,
                           n_id=node_id,
                           fib=node['flow_to_port'])
    node['device'] = FairPacketSwitch(env,
                                      k,
                                      pir,
                                      buffer_size,
                                      weights,
                                      'DRR',
                                      flow_classes,
                                      element_id=f"{node_id}")
    node['device'].demux.fib = node['flow_to_port']

for n in ft.nodes():
    node = ft.nodes[n]
    for port_number, next_hop in node['port_to_nexthop'].items():
        node['device'].ports[port_number].out = ft.nodes[next_hop]['device']

for flow_id, flow in all_flows.items():
    flow.pkt_gen.out = ft.nodes[flow.src]['device']
    ft.nodes[flow.dst]['device'].demux.ends[flow_id] = flow.pkt_sink

In [7]:
# Execute the simulation
env.run(until=100)

# Sample 5 flows to print out.
for flow_id in sample(all_flows.keys(), 5):
    print(f"Flow {flow_id}")
    print(all_flows[flow_id].pkt_sink.waits)
    print(all_flows[flow_id].pkt_sink.arrivals)
    print(all_flows[flow_id].pkt_sink.perhop_times)

Flow 68
defaultdict(<class 'list'>, {68: [0.030682934111728066, 0.03454038542741911, 0.0032299951778744784, 0.003418418539148149, 0.06613993770648996, 0.06217614526075721, 0.027285034209329595, 0.0068251656495688096, 0.11066427396256895, 0.043041972879224843, 0.2840769329166246, 0.020518216640972753, 0.035162574863637275, 0.036822681529615764, 0.03189145724120124, 0.09454258813253524, 0.1210873343042298, 0.004729041046939386, 0.0013677643987861643, 0.06913169567323152, 0.04875945404062598, 0.022466174957344265, 0.05453458168321035, 0.14826595664382225, 0.1034581024382959, 0.08218636592496154, 0.02926559649938909, 0.019987290139990677, 0.02872726241973389, 0.08554865476659756, 0.0543175593588856, 0.06004345788261389, 0.06853481087865276, 0.08693250374295758, 0.08079427025800356, 0.0815332468391361, 0.006122525862572559, 0.043754151625023496, 0.023267248183877598, 0.004704010890996813, 0.004896794172665864, 0.03952481511464612, 0.2152873817496861, 0.11324422656748112, 0.04570666182408445

/tmp/ipykernel_8312/1304441848.py:4: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  for flow_id in sample(all_flows.keys(), 5):
